# TIA Posts Ingestor

In [52]:
from pymongo import MongoClient
import requests
import json
import os
import time

In [53]:


# create client for mongo
client = MongoClient('localhost',
                     username='root',
                     password='example')

# get db from client
db = client.tia

# get collection from db
posts_collection = db.posts

In [54]:
url = 'https://www.techinasia.com/wp-json/techinasia/2.0/posts'
start_page = 1 # NOTE: maximum page only accept 30
end_page = 30 # NOTE: maximum page only accept 30
per_page = 30 # NOTE: maximum per_page only accept 30

for page in range(start_page, end_page):
    # get the posts data
    r = requests.get(url, params={'page': page, 'per_page': per_page}, headers={'User-Agent': 'Tia-ETL'})
    posts = r.json()['posts']

    # add new '_id' key to the posts data
    posts = [{'_id': post['id']} | post for post in posts]

    # NOTE: still can't find how to handle duplicate key (id) when performing bulk insert (insert_many)

    for post in posts:
        # insert per post
        posts_collection.replace_one({'_id': post['_id']}, post, upsert=True)
    time.sleep(5)